# Assignment - Scraping Post Codes

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import lxml

In [2]:
page_address = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(page_address)
print(r.status_code)
soup = bs(r.text)
#print(soup.prettify())

200


In [3]:
table = soup.find('table', class_='wikitable')

scraping the table...

In [4]:
df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
#print(table.find_all("tr"))
for row in table.find_all("tr"):
    #print('iteration')
    cells = row.findAll('td')
    if (len(cells) == 3):
        a = cells[2].text.rstrip('\n')
        if ('Not assigned' in a):
            if (not('Not assigned' in cells[1])):
                #print('append 2')
                df = df.append(pd.Series({'PostalCode':cells[0].text, 'Borough':cells[1].text, 'Neighborhood':cells[1].text + ', '}), ignore_index=True)
        else:
            a=a+', '
            #print('append 3')
            df = df.append(pd.Series({'PostalCode':cells[0].text, 'Borough':cells[1].text, 'Neighborhood':a}), ignore_index=True)
    

assembling all neighborhoods within the same Postal Code

In [5]:
dres = df.groupby(['PostalCode', 'Borough']).sum()

In [6]:
#dres.index

stripping trailing spaces and commas

In [7]:
dres['Neighborhood'] = dres['Neighborhood'].apply(lambda x: x.rstrip(', '))
#dres

The next two cells to do with the geocoder interface were completely commented out. The geocoder interface now requires paying account, and I would have to publish the key for review. Instead, the pre-locaded .csv will be used. The code is left in case someone is interested

In [8]:
#dres.reset_index(inplace=True)
#dres

In [9]:
#import geocoder 
#d = dres
#def locate(x):
#    y = x['PostalCode']
#    print(f'locating {y}')
#    lat_lng_coords = None
#    while(lat_lng_coords is None):
#        g = geocoder.google(x['PostalCode'] + ', Toronto, Ontario')
#        print(g)
#        lat_lng_coords = g.latlng
#        #print('tried')
#    x['Lattitude'] = lat_lng_coords[0]
#    x['Longitude'] = lat_lng_coords[1]
#    return x
#
#d = d.apply(locate, axis=1)
#d


In [10]:
url = 'https://cocl.us/Geospatial_data'
locs = pd.read_csv(url)
#locs.set_index(['Postal Code'], inplace=True)

In [11]:
#dres.set_index(['PostalCode'], inplace=True)
dres.reset_index(inplace=True)

In [12]:
locs


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [13]:
res = pd.merge(left=dres,right=locs, left_on='PostalCode', right_on='Postal Code')
res.drop(columns='Postal Code', inplace=True)
res

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
